# TEMPLATE DOCUMENT

This is a template document for training ASR models.
This document shows the simplest flow of ASR model with `espnetez`.

First, we need `espnet` library.

In [ ]:
%pip install espnet

### Prepare dataset

In [ ]:
import espnetez as ez

# We need to create dump file, if we don't use external dataset package.
DUMP_DIR = "./dump/"

# We need to define the data information.
data_info = {
    "speech": ["wav.scp", "sound"],
    "text": ["text", "text"],
}
ez.data.create_dump_file(DUMP_DIR, your_dataset, data_info)

### Train sentencepiece model

In [ ]:
ez.preprocess.prepare_sentences(["dump/train/text"], "dump/spm")
ez.preprocess.train_sentencepiece(
    "dump/spm/train.txt",
    "data/bpemodel",
    vocab_size=5000,
)

### Train ASR model

First we need to define the training config, and then run `collect_stats` and `train` function.

In [ ]:
# define training config
EXP_DIR = "exp/train_asr_branchformer_e24_amp"
STATS_DIR = "exp/stats"

training_config = ez.config.from_yaml(
    "asr",
    "config/train_asr_e_branchformer_size256_mlp1024_linear1024_e12_mactrue_edrop0.0_ddrop0.0.yaml",
)
preprocessor_config = ez.utils.load_yaml("config/preprocess.yaml")
training_config.update(preprocessor_config)

with open(preprocessor_config["token_list"], "r") as f:
    training_config["token_list"] = [t.replace("\n", "") for t in f.readlines()]


In [ ]:
# Define the Trainer class
trainer = ez.Trainer(
    task='asr',
    train_config=training_config,
    train_dump_dir="dump/train",
    valid_dump_dir="dump/dev",
    data_info=data_info,
    output_dir=EXP_DIR,
    stats_dir=STATS_DIR,
    ngpu=1,
)
trainer.collect_stats()
trainer.train()